In [5]:
import numpy as np
import pandas as pd

from gr_pursuer.agents.target import Target
from gr_pursuer.agents.observer import Observer

from multigrid.envs.goal_prediction import GREnv


scenarios = pd.read_csv('gr_pursuer/data/64_distance_50_0/scenarios.csv')
scenarios

,layout,scenario,observer_pos,target_pos,observer_dir,target_dir,goals,target_goal,cost,base_grid,hidden_cost,PARTITION
0,25,1,"(24, 43)","(20, 43)",2,1,"[(50, 5), (10, 44), (29, 32)]","(10, 44)","[64, 15, 16]","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",TRAIN
1,45,5,"(50, 15)","(50, 11)",3,1,"[(61, 18), (43, 27), (31, 60)]","(61, 18)","[14, 19, 64]","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",TEST
2,0,0,"(52, 50)","(52, 46)",3,1,"[(52, 47), (44, 38), (61, 38)]","(52, 47)","[3, 20, 21]","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",TRAIN
3,14,6,"(34, 11)","(34, 7)",3,1,"[(12, 50), (41, 25), (9, 6)]","(41, 25)","[61, 21, 30]","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",TRAIN
4,25,0,"(39, 49)","(39, 45)",3,1,"[(49, 27), (50, 62), (57, 24)]","(50, 62)","[32, 24, 43]","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",TRAIN
...,...,...,...,...,...,...,...,...,...,...,...,...
495,35,8,"(46, 50)","(42, 50)",2,1,"[(45, 56), (45, 35), (47, 30)]","(45, 56)","[7, 16, 21]","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",VALID
496,20,7,"(62, 15)","(58, 15)",2,1,"[(8, 22), (39, 61), (26, 13)]","(26, 13)","[61, 69, 38]","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",TRAIN
497,22,4,"(14, 5)","(14, 1)",3,1,"[(9, 51), (1, 45), (23, 46)]","(23, 46)","[51, 53, 50]","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",TRAIN
498,37,8,"(56, 20)","(52, 20)",2,1,"[(44, 6), (15, 47), (26, 2)]","(44, 6)","[26, 68, 48]","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",VALID


In [7]:
scenario = scenarios.iloc[68]

base_grid = np.array(eval(scenario["base_grid"]))
goals = eval(scenario["goals"])
hidden_cost = np.array(eval(scenario["hidden_cost"]))
layout = scenario["layout"]
nscenario = scenario["scenario"]

env = GREnv(size=32, base_grid=base_grid, 
            see_through_walls=[False, True],
            agent_view_size=[5, 3],
            goals=goals, hidden_cost=hidden_cost, 
            # enable_hidden_cost=enable_hidden_cost, 
            # render_mode='human'
)

In [2]:
import gr_pursuer

ModuleNotFoundError: No module named 'gr_pursuer'

In [3]:
import gr_pursuer

ModuleNotFoundError: No module named 'gr_pursuer'